In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import re

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 节省内存读文件
def reduce_mem_usage(df):
    """
    iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    @param df:
    @return:
    """
    start_mem = df.memory_usage().sum()
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('str')

    end_mem = df.memory_usage().sum()
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
dtype_ = {'sid': str,
          'package': str,
          'version': str,
          'android_id': str,
          'media_id': str,
          'apptype': str,
          'location': str,
          'cus_type': str,
          'carrier': str,
          'os': str,
          'osv': str,
          'lan': str}

In [4]:
train = reduce_mem_usage(pd.read_csv('train.csv', dtype=dtype_, index_col=0))
test1 = reduce_mem_usage(pd.read_csv('test1.csv', dtype=dtype_, index_col=0))

Memory usage of dataframe is 84000000.00 MB
Memory usage after optimization is: 66500000.00 MB
Decreased by 20.8%
Memory usage of dataframe is 24000000.00 MB
Memory usage after optimization is: 19800000.00 MB
Decreased by 17.5%


In [5]:
train.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,316361,1199,46000.0,0.0,0.0,0.0,1,nan,104,6.0,android,9,18,1438873,1.559893e+12,8,2135019403,0,2329670524,601
1,135939,893,0.0,0.0,0.0,0.0,1,nan,19,6.0,android,8.1,0,1185582,1.559994e+12,4,2782306428,1,2864801071,1000
2,399254,821,0.0,760.0,0.0,360.0,1,nan,559,0.0,android,8.1.0,0,1555716,1.559837e+12,0,1392806005,2,628911675,696
3,68983,1004,46000.0,2214.0,0.0,1080.0,0,nan,129,2.0,android,8.1.0,0,1093419,1.560042e+12,0,3562553457,3,1283809327,753
4,288999,1076,46000.0,2280.0,0.0,1080.0,1,zh-CN,64,2.0,android,8.0.0,0,1400089,1.559867e+12,5,2364522023,4,1510695983,582


In [6]:
test1.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,317625,1181,46000.0,2196.0,2.0,1080.0,CN,639,2.0,Android,8.1.0,188,1440682,1.559872e+12,7,1672223856,57,3872258917,658
1,435108,944,46003.0,2280.0,3.0,1080.0,zh-CN,704,6.0,Android,8.1.0,221,1606824,1.559739e+12,3,3767901757,23,129322164,943
2,0,1106,46000.0,0.0,0.0,0.0,nan,39,2.0,android,5.1,1562,1774642,1.559614e+12,0,454638703,30,4226678391,411
3,451504,761,46000.0,1344.0,0.0,720.0,nan,54,2.0,android,7.1.1,9,1742535,1.559668e+12,0,1507622951,65,3355419572,848
4,0,1001,46000.0,665.0,0.0,320.0,zh-CN,29,5.0,Android,8.1.0,4,1689686,1.559694e+12,0,4116351093,148,2644467751,411


In [7]:
def timestamp_to_str(timestamp):
    return datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S')

In [8]:
train['timestamp'] = train['timestamp'].apply(timestamp_to_str)
test1['timestamp'] = test1['timestamp'].apply(timestamp_to_str)

In [9]:
train.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,316361,1199,46000.0,0.0,0.0,0.0,1,nan,104,6.0,android,9,18,1438873,2019-06-07 15:32:01,8,2135019403,0,2329670524,601
1,135939,893,0.0,0.0,0.0,0.0,1,nan,19,6.0,android,8.1,0,1185582,2019-06-08 19:40:40,4,2782306428,1,2864801071,1000
2,399254,821,0.0,760.0,0.0,360.0,1,nan,559,0.0,android,8.1.0,0,1555716,2019-06-06 23:59:13,0,1392806005,2,628911675,696
3,68983,1004,46000.0,2214.0,0.0,1080.0,0,nan,129,2.0,android,8.1.0,0,1093419,2019-06-09 09:00:12,0,3562553457,3,1283809327,753
4,288999,1076,46000.0,2280.0,0.0,1080.0,1,zh-CN,64,2.0,android,8.0.0,0,1400089,2019-06-07 08:28:13,5,2364522023,4,1510695983,582


In [10]:
test1.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,317625,1181,46000.0,2196.0,2.0,1080.0,CN,639,2.0,Android,8.1.0,188,1440682,2019-06-07 09:42:30,7,1672223856,57,3872258917,658
1,435108,944,46003.0,2280.0,3.0,1080.0,zh-CN,704,6.0,Android,8.1.0,221,1606824,2019-06-05 20:53:56,3,3767901757,23,129322164,943
2,0,1106,46000.0,0.0,0.0,0.0,nan,39,2.0,android,5.1,1562,1774642,2019-06-04 10:07:42,0,454638703,30,4226678391,411
3,451504,761,46000.0,1344.0,0.0,720.0,nan,54,2.0,android,7.1.1,9,1742535,2019-06-05 01:03:22,0,1507622951,65,3355419572,848
4,0,1001,46000.0,665.0,0.0,320.0,zh-CN,29,5.0,Android,8.1.0,4,1689686,2019-06-05 08:15:54,0,4116351093,148,2644467751,411


In [11]:
# ID
id_feat = ['sid', 'android_id']
# 类别变量
cat_feat = ['package', 'version', 'media_id', 'apptype', 'location', 'cus_type',
            'ntt', 'carrier', 'osv', 'lan']
# 数值变量
numeric_feat = ['fea_hash', 'fea1_hash', 'dev_height', 'dev_width', 'dev_ppi']
# 时间变量
time_feat = ['timestamp']

In [12]:
def analysis(df, col):
    print(i)
    print('unique: \n', df[col].unique())
    print('value_counts: \n', df[col].value_counts())
    print('nunique: \n', df[col].nunique())
    print('-' * 20)
    print('\n')

In [13]:
for i in cat_feat:
    if i != 'package':
        analysis(train, i)
        print('-' * 10)
        analysis(test1, i)
        print('*' * 20)

version
unique: 
 ['8' '4' '0' '5' '9' '7' '3' '1' '6' '11' '2' 'v1' 'V3' 'GA3' '10'
 'P_Final_6' '15' 'V6' ' 2' 'GA2' 'V2' '50']
value_counts: 
 0            292156
5             56691
8             38348
4             24262
3             23857
7             22801
2             12750
1              9184
11             8867
6              8527
V3              806
9               765
v1              611
10              126
P_Final_6        92
V6               88
GA3              35
GA2              10
15                9
V2                8
 2                6
50                1
Name: version, dtype: int64
nunique: 
 22
--------------------


----------
version
unique: 
 ['7' '3' '0' '5' '1' '11' '2' '4' '6' '8' 'v1' 'V3' 'V6' '9' 'GA3' '10'
 '15' 'P_Final_6' 'GA2' 'V2' '50' ' 2' '20']
value_counts: 
 0            88113
5            16768
8            11410
4             7273
3             7160
7             6831
2             3733
1             2722
11            2711
6             24

Name: apptype, dtype: int64
nunique: 
 78
--------------------


********************
location
unique: 
 ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '-1' '27' '28'
 '29' '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42'
 '43' '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56'
 '57' '58' '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70'
 '71' '72' '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84'
 '85' '86' '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98'
 '99' '100' '101' '102' '103' '104' '105' '106' '107' '108' '109' '110'
 '111' '112' '113' '114' '115' '116' '117' '118' '119' '120' '121' '122'
 '123' '124' '125' '126' '127' '128' '129' '130' '131' '132' '133' '134'
 '135' '136' '137' '138' '139' '140' '141' '142' '143' '144' '145' '146'
 '147' '148' '149' '150' '151' '152' '153' '154' '155' '156' '157' '158'
 '159' '160' '161' '162' '163' '16

value_counts: 
 411     133055
810       6610
1323      6596
1057      6568
1304      6549
1076      6522
848       6501
392       6497
981       6497
1190      6495
582       6484
1000      6480
544       6474
962       6473
1152      6471
1019      6469
829       6462
905       6461
696       6459
601       6456
639       6453
1285      6453
1380      6451
1209      6449
430       6448
658       6448
772       6446
1095      6442
1228      6433
1171      6431
525       6429
1114      6426
791       6426
1247      6426
449       6425
1266      6421
715       6418
753       6413
335       6412
354       6408
886       6403
924       6401
867       6400
620       6399
468       6395
487       6393
1361      6391
316       6388
297       6383
1038      6377
677       6376
373       6373
506       6372
1342      6371
943       6366
1133      6333
563       6323
734       6319
Name: cus_type, dtype: int64
nunique: 
 58
--------------------


----------
cus_type
unique: 
 ['658' '943' '411'

 8.1.0                              45224
9                                  21460
6.0.1                               9483
7.1.1                               8098
9.0.0                               7396
6.0                                 7139
5.1                                 5799
8.1                                 5761
5.1.1                               5211
8.0.0                               4493
7.1.2                               3660
4.4.4                               3148
7.0                                 2908
Android_8.1.0                       2320
Android_9                           2089
nan                                 1992
5.0.2                               1611
7.9.2                               1350
8.0                                 1247
7.9.0                               1237
7.1                                 1107
9.0                                  546
3.8.6                                541
4.4.2                                505
6.0.0          

### os 删除

In [14]:
# os列取值唯一，删除
train.drop('os', axis=1, inplace=True)
test1.drop('os', axis=1, inplace=True)

### ntt, dev_height, dev_ppi, dev_width

In [15]:
train['ntt'].replace(0., np.nan, inplace=True)
test1['ntt'].replace(0., np.nan, inplace=True)

train['dev_height'].replace(0., np.nan, inplace=True)
test1['dev_height'].replace(0., np.nan, inplace=True)

train['dev_ppi'].replace(0., np.nan, inplace=True)
test1['dev_ppi'].replace(0., np.nan, inplace=True)

train['dev_width'].replace(0., np.nan, inplace=True)
test1['dev_width'].replace(0., np.nan, inplace=True)

### carrier 运营商

In [16]:
# 后续做独热编码
carrier_map = {'46000.0': 1,
               '0.0': np.nan,
               '46003.0': 2,
               '-1.0': np.nan,
               '46001.0': 3}

train['carrier'] = train['carrier'].map(carrier_map)
test1['carrier'] = test1['carrier'].map(carrier_map)

### lan 语言

In [17]:
lan_map = {'nan': np.nan,
           'zh-CN': 1,
           'zh': 1,
           'cn': 1,
           'zh-cn': 1,
           'zh_CN': 1,
           'Zh-CN': 1,
           'Zh-CN': 1,
           'ZH': 1,
           'en': 3,
           'CN': 1,
           'en-GB': 3,
           'tw': 2,
           'TW': 2,
           'zh-TW': 2,
           'zh_CN_#Hans': 10,
           'zh-HK': 4,
           'en-US': 5,
           'en-US': 5,
           'en_US': 5,
           'ko': 6,  # 韩国
           'zh-MO': 7,  # 澳门
           'it': 8,  # 意大利
           'mi': 9,
           'ja': 10,  # 日本
           'zh-US': np.nan,
           'in_ID': np.nan}

train['lan'] = train['lan'].map(lan_map)
test1['lan'] = test1['lan'].map(lan_map)

### version 版本

In [18]:
version_map = {'8': 8,
               '4': 4,
               '0': 0,
               '5': 5,
               '9': 9,
               '7': 7,
               '3': 3,
               '1': 1,
               '6': 6,
               '11': 11,
               '2': 2,
               'v1': 1,
               'V3': 3,
               'GA3': 3,
               '10': 10,
               'P_Final_6': 6,
               '15': 15,
               'V6': 6,
               ' 2': 2,
               'GA2': 2,
               'V2': 2,
               '50': 5,
               '20': 2}

train['version'] = train['version'].map(version_map)
test1['version'] = test1['version'].map(version_map)

### osv 处理

In [19]:
train['osv'].replace('f073b_changxiang_v01_b1b8_20180915', '1', inplace=True)
train['osv'].replace('%E6%B1%9F%E7%81%B5OS+5.0', '5', inplace=True)
# 7910, 21100, 21000, 7930, 71200, 7920
patt_7 = re.compile('7910|7930|71200|7920|71300')
train['osv'].replace(patt_7, '7', inplace=True)
patt_2 = re.compile('21100|21000')
train['osv'].replace(patt_2, '2', inplace=True)

# GIONEE_YNGA
test1['osv'].replace('GIONEE_YNGA', 'nan', inplace=True)
test1['osv'].replace('12.0', '11', inplace=True)

# 7910, 21100, 21000, 7930, 71200, 7920
test1['osv'].replace(patt_7, '7', inplace=True)
test1['osv'].replace(patt_2, '2', inplace=True)


def osv_process(s): 
    res = re.split('\.', s)[0]
    if s == 'nan':
        return res
    elif res in [str(i) for i in range(1, 13)]:
        return int(res)
    else:
        res2 = re.split('\_|\ ', res)[-1]
        if res2 not in [str(i) for i in range(1, 13)]:
            print(res2)
        return int(res2)

print('train')    
train['osv'] = train['osv'].apply(osv_process)
print('test')
test1['osv'] = test1['osv'].apply(osv_process)

train['osv'].replace('nan', np.nan, inplace=True)
test1['osv'].replace('nan', np.nan, inplace=True)

train
test


In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 19 columns):
android_id    500000 non-null object
apptype       500000 non-null object
carrier       435093 non-null float64
dev_height    392986 non-null float16
dev_ppi       116709 non-null float16
dev_width     392989 non-null float16
label         500000 non-null int8
lan           316720 non-null float64
media_id      500000 non-null object
ntt           479383 non-null float16
osv           493439 non-null float64
package       500000 non-null object
sid           500000 non-null object
timestamp     500000 non-null object
version       500000 non-null int64
fea_hash      500000 non-null object
location      500000 non-null object
fea1_hash     500000 non-null int64
cus_type      500000 non-null object
dtypes: float16(4), float64(3), int64(2), int8(1), object(9)
memory usage: 61.5+ MB


In [21]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 0 to 149999
Data columns (total 18 columns):
android_id    150000 non-null object
apptype       150000 non-null object
carrier       130481 non-null float64
dev_height    117801 non-null float16
dev_ppi       34667 non-null float16
dev_width     117801 non-null float16
lan           94451 non-null float64
media_id      150000 non-null object
ntt           143818 non-null float16
osv           148007 non-null float64
package       150000 non-null object
sid           150000 non-null object
timestamp     150000 non-null object
version       150000 non-null int64
fea_hash      150000 non-null object
location      150000 non-null object
fea1_hash     150000 non-null int64
cus_type      150000 non-null object
dtypes: float16(4), float64(3), int64(2), object(9)
memory usage: 18.3+ MB


### fea_hash 脏数据处理

In [22]:
train['fea_hash'] = train['fea_hash'].astype(str)
test1['fea_hash'] = test1['fea_hash'].astype(str)

In [23]:
train['fea_hash'] = train['fea_hash'].apply(lambda x: np.nan if ':' in x else int(x))
test1['fea_hash'] = test1['fea_hash'].apply(lambda x: np.nan if ':' in x else int(x))

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 19 columns):
android_id    500000 non-null object
apptype       500000 non-null object
carrier       435093 non-null float64
dev_height    392986 non-null float16
dev_ppi       116709 non-null float16
dev_width     392989 non-null float16
label         500000 non-null int8
lan           316720 non-null float64
media_id      500000 non-null object
ntt           479383 non-null float16
osv           493439 non-null float64
package       500000 non-null object
sid           500000 non-null object
timestamp     500000 non-null object
version       500000 non-null int64
fea_hash      499910 non-null float64
location      500000 non-null object
fea1_hash     500000 non-null int64
cus_type      500000 non-null object
dtypes: float16(4), float64(4), int64(2), int8(1), object(8)
memory usage: 61.5+ MB


In [25]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 0 to 149999
Data columns (total 18 columns):
android_id    150000 non-null object
apptype       150000 non-null object
carrier       130481 non-null float64
dev_height    117801 non-null float16
dev_ppi       34667 non-null float16
dev_width     117801 non-null float16
lan           94451 non-null float64
media_id      150000 non-null object
ntt           143818 non-null float16
osv           148007 non-null float64
package       150000 non-null object
sid           150000 non-null object
timestamp     150000 non-null object
version       150000 non-null int64
fea_hash      149977 non-null float64
location      150000 non-null object
fea1_hash     150000 non-null int64
cus_type      150000 non-null object
dtypes: float16(4), float64(4), int64(2), object(8)
memory usage: 18.3+ MB


### 导出成h5文件

In [26]:
train.to_hdf('train.h5', 'df')
test1.to_hdf('test1.h5', 'df')